In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline


df = pd.read_csv('./listings.csv')
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


In [90]:
df_cal = pd.read_csv('./calendar.csv')
df_cal.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [91]:
#Sort the list and drop na
df_cal_no_na = df_cal.sort_values(by=["listing_id","date"]).dropna()
df_cal_no_na.head()

,listing_id,date,available,price
797216,3335,2016-02-29,t,$120.00
797217,3335,2016-03-01,t,$120.00
797218,3335,2016-03-02,t,$120.00
797219,3335,2016-03-03,t,$120.00
797220,3335,2016-03-04,t,$120.00


In [92]:
def clean_price_field(df, fieldname):
    '''
    INPUT:
    df - pandas dataframe containing price column
    fieldname - price column name in dataframe which should be cleaned up to get a float value for analysis
    
    OUTPUT:
    df - same dataframe that has price column without '$' or ','. Price column will be a float object
    '''
    df[fieldname] = [x.strip('$') for x in df[fieldname]]
    df[fieldname] = [x.replace(',','') for x in df[fieldname]]
    df[fieldname] = df[fieldname].astype(float)
    return df[fieldname]

In [93]:
df_cal_no_na['price'] = clean_price_field(df_cal_no_na, "price")
df_cal_no_na.head()

,listing_id,date,available,price
797216,3335,2016-02-29,t,120
797217,3335,2016-03-01,t,120
797218,3335,2016-03-02,t,120
797219,3335,2016-03-03,t,120
797220,3335,2016-03-04,t,120


In [94]:
def get_seasonal_price(df):
    '''
    INPUT:
    df - pandas dataframe for calendar with columns listing_id, date and price
    
    
    OUTPUT:
    df - new dataframe that has following features:
        1. columns in the new dataframe include listing_id, season, price
        2. season is calculated based on the dates - Dec to Feb = Winter, Mar to May = Spring, 
                                                        Jun to Aug = Summer, Sep to Nov = Fall
        3. Seasonal price is calculated by taking mean of the price in a particular season for a listing id
    '''
    prev_listing_id = 0
    winter_sum = spring_sum= summer_sum= fall_sum= winter_count= spring_count= summer_count= fall_count = 0 
    calendar_data = []
    for index, row in df.iterrows():
        if prev_listing_id != row['listing_id'] and prev_listing_id != 0:
            calendar_data.append([prev_listing_id, 'winter', winter_sum/winter_count if winter_count else 0])
            calendar_data.append([prev_listing_id, 'spring', spring_sum/spring_count if spring_count else 0])
            calendar_data.append([prev_listing_id, 'summer', summer_sum/summer_count if summer_count else 0])
            calendar_data.append([prev_listing_id, 'fall', fall_sum/fall_count if fall_count else 0])
            winter_sum = spring_sum= summer_sum= fall_sum= winter_count= spring_count= summer_count= fall_count = 0 
        elif row['date'] >="2016-01-01" and row['date'] <="2016-02-29":
            winter_sum += row['price']
            winter_count +=1
        elif row['date'] >="2016-03-01" and row['date'] <="2016-05-31":
            spring_sum += row['price']
            spring_count += 1
        elif row['date'] >="2016-06-01" and row['date'] <="2016-08-31":
            summer_sum += row['price']
            summer_count += 1
        elif row['date'] >="2016-09-01" and row['date'] <="2016-11-30":
            fall_sum += row['price']
            fall_count += 1
        elif row['date'] >="2016-12-01" and row['date'] <="2016-12-31":
            winter_sum += row['price']
            winter_count += 1
        prev_listing_id = row['listing_id']
    calendar_data.append([prev_listing_id, 'winter', winter_sum/winter_count if winter_count else 0])
    calendar_data.append([prev_listing_id, 'spring', spring_sum/spring_count if spring_count else 0])
    calendar_data.append([prev_listing_id, 'summer', summer_sum/summer_count if summer_count else 0])
    calendar_data.append([prev_listing_id, 'fall', fall_sum/fall_count if fall_count else 0])
    df_new_calendar = pd.DataFrame(calendar_data,columns=['listing_id','season', 'price'])
    return df_new_calendar
            

In [95]:
df_new_calendar = get_seasonal_price(df_cal_no_na)
df_new_calendar.head(15)

,listing_id,season,price
0,3335,winter,120.000000
1,3335,spring,120.000000
2,3335,summer,120.000000
3,3335,fall,120.000000
4,4291,winter,82.000000
5,4291,spring,82.000000
6,4291,summer,82.000000
7,4291,fall,82.000000
8,5682,winter,50.333333
9,5682,spring,54.491803


In [96]:
df_new_calendar.rename(columns={'price':'seasonal_price'}, inplace=True)
df_new_calendar.head()

,listing_id,season,seasonal_price
0,3335,winter,120
1,3335,spring,120
2,3335,summer,120
3,3335,fall,120
4,4291,winter,82


In [97]:
#Merge the listing dataframe and the new calendar data frame
df.rename(columns={'id':'listing_id'}, inplace=True) #changing the column name to merge on this column
df_merged = pd.merge(df_new_calendar, df, on="listing_id")
df_merged.shape

(14892, 94)

In [98]:
df_merged.head()

,listing_id,season,seasonal_price,listing_url,scrape_id,last_scraped,name,summary,space,description,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,3335,winter,120,https://www.airbnb.com/rooms/3335,20160104002432,2016-01-04,Sweet Seattle Urban Homestead 2 Bdr,Welcome! If you stay here you will be living i...,Welcome! Come enjoy your time in Seattle at a...,Welcome! If you stay here you will be living i...,...,NaN,f,NaN,WASHINGTON,f,strict,f,f,4,NaN
1,3335,spring,120,https://www.airbnb.com/rooms/3335,20160104002432,2016-01-04,Sweet Seattle Urban Homestead 2 Bdr,Welcome! If you stay here you will be living i...,Welcome! Come enjoy your time in Seattle at a...,Welcome! If you stay here you will be living i...,...,NaN,f,NaN,WASHINGTON,f,strict,f,f,4,NaN
2,3335,summer,120,https://www.airbnb.com/rooms/3335,20160104002432,2016-01-04,Sweet Seattle Urban Homestead 2 Bdr,Welcome! If you stay here you will be living i...,Welcome! Come enjoy your time in Seattle at a...,Welcome! If you stay here you will be living i...,...,NaN,f,NaN,WASHINGTON,f,strict,f,f,4,NaN
3,3335,fall,120,https://www.airbnb.com/rooms/3335,20160104002432,2016-01-04,Sweet Seattle Urban Homestead 2 Bdr,Welcome! If you stay here you will be living i...,Welcome! Come enjoy your time in Seattle at a...,Welcome! If you stay here you will be living i...,...,NaN,f,NaN,WASHINGTON,f,strict,f,f,4,NaN
4,4291,winter,82,https://www.airbnb.com/rooms/4291,20160104002432,2016-01-04,Sunrise in Seattle Master Suite,NaN,"Located in Seattle, this is a spacious, clean...","Located in Seattle, this is a spacious, clean...",...,9,f,NaN,WASHINGTON,f,moderate,f,f,5,1.14


In [99]:
#Selecting numerical fields from the dataframe for our analysis
df_numeric_vals = df_merged[["review_scores_rating", "review_scores_accuracy","review_scores_cleanliness",
         "review_scores_checkin","review_scores_communication","review_scores_location",
                      "review_scores_value","accommodates", "seasonal_price"]]
df_numeric_vals.shape

(14892, 9)

In [120]:
#Selecting categorial fields from the dataframe for our analysis
df_category = df_merged[["season", "neighbourhood_group_cleansed","room_type","property_type"]]
df_category.shape

(14892, 4)

In [101]:
def create_dummy_df(df, cat_cols, dummy_na):
    '''
    INPUT:
    df - pandas dataframe with categorical variables you want to dummy
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains all columns that were not specified as categorical
            2. removes all the original columns in cat_cols
            3. dummy columns for each of the categorical columns in cat_cols
            4. if dummy_na is True - it also contains dummy columns for the NaN values
            5. Use a prefix of the column name with an underscore (_) for separating 
    '''  
    df = pd.get_dummies(df,columns=cat_cols,dummy_na=dummy_na,prefix_sep='_');
    return df


In [121]:
#Getting all the categorical dummy fields for our analysis
df_category_new = create_dummy_df(df_category, ["neighbourhood_group_cleansed","room_type","season","property_type"], True)
df_category_new.shape

(14892, 44)

In [122]:
#Concatenating the selected numerical and categorical fields
df_final = pd.concat([df_numeric_vals, df_category_new], axis=1)
df_final.shape

(14892, 53)

In [123]:
#Handling NaNs

# Dropping any NaNs
df_final = df_final.dropna()
df_final.head()

#Fill mean for NaNs
# fill_mean = lambda col: col.fillna(col.mean())
# df_final = df_final.apply(fill_mean, axis=0)

,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,accommodates,seasonal_price,neighbourhood_group_cleansed_Ballard,...,property_type_Condominium,property_type_Dorm,property_type_House,property_type_Loft,property_type_Other,property_type_Tent,property_type_Townhouse,property_type_Treehouse,property_type_Yurt,property_type_nan
4,92,10,9,10,9,9,9,2,82.000000,0,...,0,0,1,0,0,0,0,0,0,0
5,92,10,9,10,9,9,9,2,82.000000,0,...,0,0,1,0,0,0,0,0,0,0
6,92,10,9,10,9,9,9,2,82.000000,0,...,0,0,1,0,0,0,0,0,0,0
7,92,10,9,10,9,9,9,2,82.000000,0,...,0,0,1,0,0,0,0,0,0,0
8,96,10,10,10,10,9,10,2,50.333333,0,...,0,0,0,0,0,0,0,0,0,0


In [132]:
#Using the df_final dataframe having our key features to predict seasonal price
X = df_final.drop("seasonal_price", axis=1);
y = df_final['seasonal_price']

#Four steps:
#Mean function
# fill_mean = lambda col: col.fillna(col.mean())
# # Fill the mean
# X = X.apply(fill_mean, axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)

#Instantiate
lm_model = LinearRegression(normalize=True) 

#Fit - why does this break?
lm_model.fit(X_train, y_train) 

#Predict
#Score
#Predict and score the model
y_test_preds = lm_model.predict(X_test) 
"The r-squared score for your model was {} on {} values.".format(r2_score(y_test, y_test_preds), len(y_test))
# "Predicted {} vs actual {}".format(y_test_preds, [y_test])


'The r-squared score for your model was 0.398979597694 on 3706 values.'